In [5]:
import tensorflow as tf
import csv
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
! conda install -c anaconda word2vec
import matplotlib.pyplot as plt
import nltk
import string
import seaborn as sns
import re
import codecs

Solving environment: ...working... done

# All requested packages already installed.



In [6]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

pd.set_option("display.max_colwidth",300)

In [7]:
train = pd.read_csv(r"C:\Users\rnik\Sentiment Analysis\train_E6oV3lV.csv")
test = pd.read_csv(r"C:\Users\rnik\Sentiment Analysis\test_tweets_anuFYb8.csv")

In [8]:
train.shape, test.shape

((31962, 3), (17197, 2))

In [9]:
train.head(5)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,0,factsguide: society now #motivation


In [10]:
test.head(5)

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedication #willpower to find #newmaterialsâ¦
1,31964,@user #white #supremacists want everyone to see the new â #birdsâ #movie â and hereâs why
2,31965,safe ways to heal your #acne!! #altwaystoheal #healthy #healing!!
3,31966,"is the hp and the cursed child book up for reservations already? if yes, where? if no, when? ððð #harrypotter #pottermore #favorite"
4,31967,"3rd #bihday to my amazing, hilarious #nephew eli ahmir! uncle dave loves you and missesâ¦"


In [11]:
train["label"].value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [12]:
combi = train.append(test, ignore_index=True)
combi.shape

(49159, 3)

In [13]:
combi.tail(5)

,id,label,tweet
49154,49155,NaN,thought factory: left-right polarisation! #trump #uselections2016 #leadership #politics #brexit #blm &gt;3
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverready #formal #wedding #gown #dresses #mermaid â¦
49156,49157,NaN,"#hillary #campaigned today in #ohio((omg)) &amp; used words like ""assets&amp;liability"" never once did #clinton say thee(word) #radicalization"
49157,49158,NaN,"happy, at work conference: right mindset leads to culture-of-development organizations #work #mindset"
49158,49159,NaN,"my song ""so glad"" free download! #shoegaze #newmusic #newsong"


In [15]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

In [16]:
combi['clean_tweet'] = np.vectorize(remove_pattern)(combi['tweet'], "@[\w]*") 
combi.head()

,id,label,tweet,clean_tweet
0,1,0.0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0.0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,0.0,factsguide: society now #motivation,factsguide: society now #motivation


In [17]:
combi['clean_tweet'] = combi['clean_tweet'].str.replace("[^a-zA-Z#]", " ")
combi.head(10)

,id,label,tweet,clean_tweet
0,1,0.0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction #run
1,2,0.0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can t use cause they don t offer wheelchair vans in pdx #disapointed #getthanked
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love u take with u all the time in ur
4,5,0.0,factsguide: society now #motivation,factsguide society now #motivation
5,6,0.0,[2/2] huge fan fare and big talking before they leave. chaos and pay disputes when they get there. #allshowandnogo,huge fan fare and big talking before they leave chaos and pay disputes when they get there #allshowandnogo
6,7,0.0,@user camping tomorrow @user @user @user @user @user @user @user dannyâ¦,camping tomorrow danny
7,8,0.0,the next school year is the year for exams.ð¯ can't think about that ð­ #school #exams #hate #imagine #actorslife #revolutionschool #girl,the next school year is the year for exams can t think about that #school #exams #hate #imagine #actorslife #revolutionschool #girl
8,9,0.0,we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers â¦,we won love the land #allin #cavs #champions #cleveland #clevelandcavaliers
9,10,0.0,@user @user welcome here ! i'm it's so #gr8 !,welcome here i m it s so #gr


In [18]:
combi['clean_tweet'] = combi['clean_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
combi.head()

,id,label,tweet,clean_tweet
0,1,0.0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when father dysfunctional selfish drags kids into dysfunction #run
1,2,0.0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks #lyft credit cause they offer wheelchair vans #disapointed #getthanked
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model love take with time
4,5,0.0,factsguide: society now #motivation,factsguide society #motivation


In [19]:
#tokenize
tokenized_tweet = combi['clean_tweet'].apply(lambda x: x.split())
tokenized_tweet.head(10)

0                                                      [when, father, dysfunctional, selfish, drags, kids, into, dysfunction, #run]
1                                          [thanks, #lyft, credit, cause, they, offer, wheelchair, vans, #disapointed, #getthanked]
2                                                                                                           [bihday, your, majesty]
3                                                                                                  [#model, love, take, with, time]
4                                                                                                [factsguide, society, #motivation]
5                                   [huge, fare, talking, before, they, leave, chaos, disputes, when, they, there, #allshowandnogo]
6                                                                                                        [camping, tomorrow, danny]
7    [next, school, year, year, exams, think, about, that, #school, #exams, 

In [15]:
"""from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# lemmatizing
tokenized_tweet = tokenized_tweet.apply(lambda x: [lemmatizer.lemmatize(i) for i in x])
"""

'from nltk.stem import WordNetLemmatizer\nlemmatizer = WordNetLemmatizer()\n\n# lemmatizing\ntokenized_tweet = tokenized_tweet.apply(lambda x: [lemmatizer.lemmatize(i) for i in x])\n'

In [16]:
"""lemmatizer.lemmatize("camping")

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-16-c2762cd81f2a>, line 1)

In [20]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming

In [21]:
stemmer.stem("running")

'run'

In [22]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
    
combi['clean_tweet'] = tokenized_tweet

combi['clean_tweet'].head(10)

0                                                      when father dysfunct selfish drag kid into dysfunct #run
1                                        thank #lyft credit caus they offer wheelchair van #disapoint #getthank
2                                                                                           bihday your majesti
3                                                                                    #model love take with time
4                                                                                      factsguid societi #motiv
5                                    huge fare talk befor they leav chao disput when they there #allshowandnogo
6                                                                                           camp tomorrow danni
7    next school year year exam think about that #school #exam #hate #imagin #actorslif #revolutionschool #girl
8                                                   love land #allin #cav #champion #cleveland #clevelan

In [23]:
from gensim.models import Word2Vec

In [24]:
tokenized_tweet = combi['clean_tweet'].apply(lambda x: x.split()) # tokenizing

model_w2v = Word2Vec(tokenized_tweet, size=200,  window=5)
model_w2v.save("word2vec.model")

In [25]:
model_w2v = Word2Vec.load("word2vec.model")
model_w2v.train(tokenized_tweet, total_examples= len(combi['clean_tweet']), epochs=20)

(5946585, 7536020)

In [26]:
model_w2v.wv.most_similar(positive="dinner")

[('breakfast', 0.7644131183624268),
 ('lunch', 0.7624712586402893),
 ('wine', 0.7508099675178528),
 ('brunch', 0.7126549482345581),
 ('pool', 0.6884925365447998),
 ('#wine', 0.6822792887687683),
 ('dess', 0.6720693707466125),
 ('chill', 0.660775363445282),
 ('dine', 0.6458286046981812),
 ('maui', 0.6451388597488403)]

In [27]:
model_w2v.wv.most_similar(positive="trump")

[('hillari', 0.7745310068130493),
 ('suppoer', 0.7565606832504272),
 ('donald', 0.7226791381835938),
 ('voter', 0.7220083475112915),
 ('obama', 0.7157222032546997),
 ('putin', 0.7155409455299377),
 ('berni', 0.7117952108383179),
 ('obviou', 0.7033720016479492),
 ('unfit', 0.7011762261390686),
 ('campaign', 0.6828374266624451)]

In [28]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size))
            count += 1.
        except KeyError: # handling the case where the token is not in vocabulary
                         
            continue
    if count != 0:
        vec /= count
    return vec

In [29]:
wordvec_arrays = np.zeros((len(tokenized_tweet), 200))

for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet[i], 200)
    
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(49159, 200)

In [30]:
from sklearn.model_selection import train_test_split


train_w2v = wordvec_df.iloc[:31962,:]
test_w2v = wordvec_df.iloc[31962:,:]

_, _, ytrain, yvalid = train_test_split(train_w2v, train['label'],  
                                                          random_state=42, 
                                                          test_size=0.3)



print(train_w2v.shape, train['label'].shape)

xtrain_w2v = train_w2v.iloc[ytrain.index,:]
xvalid_w2v = train_w2v.iloc[yvalid.index,:]

(31962, 200) (31962,)


In [31]:
! ls -a

'ls' is not recognized as an internal or external command,
operable program or batch file.
